In [1]:
import pandas as pd
import requests
import json
import glob
import os
import urllib3

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

## GET AUTHORISATION TOKEN
Grab your token and your client ID used to request an authorisation token (https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token) and input them into the payload

In [2]:
# Start the session
session = requests.Session()

# Create the payload
payload = {
    "token": "eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1UVkdNRGxETUVaRk1qWkdOVU0yT0RnNFF6VTROalZCTlRsR1FrUTRSamM1TVROQ09UWTJOZyJ9.eyJodHRwOi8vbG9naW4ubmV3c2NvcnBhdXN0cmFsaWEuY29tLmF1L3Byb2ZpbGUiOnsidGhpbmtfaWQiOiIxNDEyMzIzMSIsImN1c3RvbWVyX3Byb2R1Y3RfaG9sZGluZyI6WyJaWl9OTF9SRUciLCJaWl9TQ19EREFGTCIsIlpaX1NDX0RERlJOT0VOVCJdLCJzaXRlIjoiSGVyYWxkU3VuIiwiaGFzX2NvbnNlbnRlZF90b190YyI6dHJ1ZSwiYXV0aFByb3ZpZGVyIjoiYXV0aDAifSwibmlja25hbWUiOiJNYXJrIiwibmFtZSI6Ik1hcmsiLCJwaWN0dXJlIjoiaHR0cHM6Ly9ncmFwaC5mYWNlYm9vay5jb20vNzE1NjE3NzU2L3BpY3R1cmUiLCJ1cGRhdGVkX2F0IjoiMjAyMS0wMS0xNFQwNzo0MDoyMi41NjlaIiwiaXNzIjoiaHR0cHM6Ly9sb2dpbi5uZXdzY29ycGF1c3RyYWxpYS5jb20vIiwic3ViIjoiYXV0aDB8YzVjZWVjOWItYWQ4NC00NWE5LWFlNWMtOTBiNjc1YzRiMWMxIiwiYXVkIjoiWllDb3RsaWhxYUd1YXFTc1N2dTBMMnZ4RGRRWEN3MTYiLCJpYXQiOjE2MTA2MTAwMjIsImV4cCI6MTYxNzk1NDAyMiwiYXRfaGFzaCI6IjdhREozd3EtOEtkT1V3cWRNNnhnMlEiLCJub25jZSI6Imppbk95ck90aFp5cmdqNXBsdVBpbk9VTkZhMHguTlFxIn0.gRo-tEGXBwqE_zXMid_wyLN_-RJtRfAO_LUwVnJRS87nTrOpEZnbApZJIgZpLlww7Mm5H_hjm3byjiPMGeHJ1zt_xmUGc0wj6IGEC9QA_Q4oP7sKjeDNEVnJJbjTSKNrkM9VQ6vkoQo-kWXHfEMp-b1gkvDiUgJ53wl3g20SZcy8L0KJ-TaiEITAsgCf2UL7N9gu0FEcjygSpthhDedVuS2xayLi8xiwx1U97nC08sXgqHSyJe4PAUlU_Th7UROLbtlDgrzOpRhHUtSmVGpalUACHpY6i4g9hvVTYOH0p_NfF9KtZbt1ALv4rHQNxHD0gr3GdHG4nuziOMvVqxKenA",
    "client_id": "Lf7m0371XCbMBlQ0fAFoGRJlfCs2JZpYvLU1uEvd",
    "client_secret": "",
    "grant_type": "social",
    "service": "auth0",
}


# Post the payload to the site to log in
s = session.post("https://supercoach.heraldsun.com.au/2020/api/afl/classic/v1/access_token", data=payload, verify=False)
display(s.text)
access_token = json.loads(s.text)['access_token']

headers = {
    "Authorization": "Bearer " + access_token
}

'{"token_type":"Bearer","expires_in":604800,"access_token":"90ee32f2e5a5eb0d35ad7aef0ee6d030562666d8","refresh_token":"6c0c0fa030978a662efe5caf14c0d82a0db43ac2"}'

## PLAYER LIST

In [3]:
def get_position_from_json(positions):
    positions_string = ''
    for position in positions:
        if len(positions_string) > 0:
            positions_string = positions_string + ' ' + position['position']
        else:
            positions_string = positions_string + position['position']
    return positions_string

In [4]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/players-cf?embed=positions", headers=headers)
player_json = json.loads(s.text)
df_player_list = pd.DataFrame()

for player in player_json:
    positions_string = get_position_from_json(player['positions'])
    df_line = pd.DataFrame(
        {
            'player_id': [player['id']],
            'feed_id': [player['feed_id']],
            'first_name': [player['first_name']],
            'last_name': [player['last_name']],
            'team': [player['team']['abbrev']],
            'position': [pos for pos in [player['positions']]],
            'position': positions_string
        }
    )
    df_player_list = pd.concat([df_player_list, df_line])
df_player_list

,player_id,feed_id,first_name,last_name,team,position
0,151,293957,Brodie,Grundy,COL,RUC
0,469,290528,Max,Gawn,MEL,RUC
0,755,295467,Jack,Macrae,WBD,MID
0,76,293535,Lachie,Neale,BRL,MID
0,244,291570,Nat,Fyfe,FRE,MID
...,...,...,...,...,...,...
0,821,1016097,Henry,Crauford,ESS,RUC
0,822,1020224,Tom,Hird,ESS,MID
0,825,297668,Nicholas,Reid,WCE,FWD
0,208,997846,Conor,*** McKenna,ESS,DEF


## LADDER AND FIXTURE DATA
- also saves each json file

In [6]:
# specify round
round_number = 18
match_json_list = []

for rd in range(1, round_number + 1):
    s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/ladderAndFixtures?round={}&scores=true".format(rd), headers=headers)  
    match_json = json.loads(s.text)
    match_json_list.append(match_json)
    with open("outputs/json files/match_json_rd{}.json".format(rd), 'w') as outfile:
        json.dump(match_json, outfile, indent=4, sort_keys=True)

## COACH IDs AND NAMES

In [6]:
df_coaches = pd.DataFrame()
first_match_json = match_json_list[0]

for coach in first_match_json['ladder']:
    df_line = pd.DataFrame(
        {
            'coach_id': [coach['userTeam']['user']['id']],
            'coach_team_id': [coach['userTeam']['id']],
            'coach_first_name': [coach['userTeam']['user']['first_name']],
            'coach_team_name': [coach['userTeam']['teamname']],
        }
    )
    df_coaches = pd.concat([df_coaches, df_line])
    
df_coaches

,coach_id,coach_team_id,coach_first_name,coach_team_name
0,86078,9806,Lester,HoweIMetYourMother
0,180650,33301,Jordan,"What's Up, Doch?"
0,45168,5524,Luke,The Dragon's Laird
0,300791,182,Paul,Coronaviney
0,21562,186,Mark,GetRichOrDieTryin
0,10857,189,Anthony,The Full Bonty
0,71052,9867,James,Bad Boys For Fyfe
0,8258,9856,Simon,It's always Sonny


In [7]:
df_coaches.to_csv('inputs/coach_list.csv')

## PLAYER MATCH RESULTS

Function to get player scores for each match

In [8]:
def get_player_score_data(scores, on_field_status, coach_id, coach_team_id, coach_first_name, coach_team_name):
    df_player_results = pd.DataFrame()
    df_player_line = pd.DataFrame()
    for player in scores:
        player_id = player['player_id'],
        if on_field_status == True:
            points = player['points']
        else:
            points = player['player']['player_stats'][0]['points']
        played_position = player['position']
        df_player_line = pd.DataFrame(
            {
                'round': rd_number,
                'coach_id': coach_id,
                'coach_team_id': coach_team_id,
                'coach_first_name': coach_first_name,
                'coach_team_name': coach_team_name,
                'player_id': player_id,
                'played_position': played_position,
                'points': points,
                'on_field': on_field_status
            })
        df_player_results = pd.concat([df_player_results, df_player_line])
    return df_player_results

In [9]:
df_player_match_results = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        rd_number = coach['round']
        coach_id = coach['userTeam']['user']['id'],
        coach_team_id = coach['userTeam']['id'],
        coach_first_name = coach['userTeam']['user']['first_name'],
        coach_team_name = coach['userTeam']['teamname'],
        df_on_field_players = get_player_score_data(coach['userTeam']['scores']['scoring'], True, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_off_field_players = get_player_score_data(coach['userTeam']['scores']['nonscoring'], False, coach_id, coach_team_id, coach_first_name, coach_team_name)
        df_all_players = pd.concat([df_on_field_players, df_off_field_players])
        df_player_match_results = pd.concat([df_player_match_results, df_all_players])

df_player_match_results = df_player_match_results.merge(df_player_list, on='player_id')
df_player_match_results['round_number_alt'] = "R" + df_player_match_results['round'].astype('str')
df_player_match_results

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt
0,1,86078,9806,Lester,HoweIMetYourMother,153,DEF,156,True,291313,Jeremy,Howe,COL,DEF,R1
1,2,86078,9806,Lester,HoweIMetYourMother,153,DEF,126,True,291313,Jeremy,Howe,COL,DEF,R2
2,3,86078,9806,Lester,HoweIMetYourMother,153,DEF,88,True,291313,Jeremy,Howe,COL,DEF,R3
3,4,86078,9806,Lester,HoweIMetYourMother,153,DEF,112,True,291313,Jeremy,Howe,COL,DEF,R4
4,1,86078,9806,Lester,HoweIMetYourMother,789,DEF,152,True,240283,Shannon,Hurn,WCE,DEF,R1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,18,71052,9867,James,Bad Boys For Fyfe,65,DEF,57,False,291548,Ryan,Lester,BRL,DEF,R18
3164,16,300791,182,Paul,Coronaviney,804,DEF,50,True,997823,Josh,Rotham,WCE,DEF,R16
3165,17,300791,182,Paul,Coronaviney,804,DEF,31,True,997823,Josh,Rotham,WCE,DEF,R17
3166,18,300791,182,Paul,Coronaviney,804,DEF,45,False,997823,Josh,Rotham,WCE,DEF,R18


In [10]:
df_player_match_results.to_csv('outputs/player_match_results.csv')

## LADDER

In [11]:
df_ladder = pd.DataFrame()

for match in match_json_list:
    for coach in match['ladder']:
        df_line = pd.DataFrame(
            {
                'round': coach['round'],
                'coach_id': [coach['userTeam']['user']['id']],
                'coach_team_id': [coach['userTeam']['id']],
                'coach_first_name': [coach['userTeam']['user']['first_name']],
                'coach_team_name': [coach['userTeam']['teamname']],
                'wins': coach['wins'],
                'draws': coach['draws'],
                'losses': coach['losses'],
                'points_for': coach['points_for'],
                'points_against': coach['points_against'],
                'league_points': coach['points'],
                'position': coach['position']
            }
        )
        df_ladder = pd.concat([df_ladder, df_line])
df_ladder

,round,coach_id,coach_team_id,coach_first_name,coach_team_name,wins,draws,losses,points_for,points_against,league_points,position
0,1,86078,9806,Lester,HoweIMetYourMother,1,0,0,1827,1476,4,1
0,1,180650,33301,Jordan,"What's Up, Doch?",1,0,0,1721,1623,4,2
0,1,45168,5524,Luke,The Dragon's Laird,1,0,0,1720,1662,4,3
0,1,300791,182,Paul,Coronaviney,1,0,0,1686,1657,4,4
0,1,21562,186,Mark,GetRichOrDieTryin,0,0,1,1657,1686,0,5
...,...,...,...,...,...,...,...,...,...,...,...,...
0,18,86078,9806,Lester,HoweIMetYourMother,10,0,7,28263,27920,40,4
0,18,300791,182,Paul,Coronaviney,10,0,7,27298,26972,40,5
0,18,45168,5524,Luke,The Dragon's Laird,8,0,9,28186,27897,32,6
0,18,71052,9867,James,Bad Boys For Fyfe,5,0,12,25905,28087,20,7


In [12]:
df_ladder.to_csv('outputs/ladder.csv')

## FIXTURE RESULTS

In [13]:
df_fixture_results = pd.DataFrame()
for match in match_json_list:
    for result in match['fixtures']:
        df_line = pd.DataFrame(
            {
                'round_number': [result['round']],
                'match_name': '{} vs {}'.format(result['user_team1']['user']['first_name'], result['user_team2']['user']['first_name']),
                'home_team_id': [result['user_team1']['id']],
                'home_team_name': [result['user_team1']['teamname']],
                'home_team_coach': [result['user_team1']['user']['first_name']],
                'home_team_points': [result['user_team1']['stats'][0]['points']],
                'away_team_id': [result['user_team2']['id']],
                'away_team_name': [result['user_team2']['teamname']],
                'away_team_coach': [result['user_team2']['user']['first_name']],
                'away_team_points': [result['user_team2']['stats'][0]['points']],
            }
        )
        df_fixture_results = pd.concat([df_fixture_results, df_line])
    
df_fixture_results

,round_number,match_name,home_team_id,home_team_name,home_team_coach,home_team_points,away_team_id,away_team_name,away_team_coach,away_team_points
0,1,Paul vs Mark,182,Coronaviney,Paul,1686,186,GetRichOrDieTryin,Mark,1657
0,1,Anthony vs Luke,189,The Full Bonty,Anthony,1662,5524,The Dragon's Laird,Luke,1720
0,1,Lester vs Simon,9806,HoweIMetYourMother,Lester,1827,9856,It's always Sonny,Simon,1476
0,1,James vs Jordan,9867,Bad Boys For Fyfe,James,1623,33301,"What's Up, Doch?",Jordan,1721
0,2,Paul vs Anthony,182,Coronaviney,Paul,1676,189,The Full Bonty,Anthony,1653
...,...,...,...,...,...,...,...,...,...,...
0,17,Jordan vs Simon,33301,"What's Up, Doch?",Jordan,1435,9856,It's always Sonny,Simon,1722
0,18,Paul vs Lester,182,Coronaviney,Paul,1843,9806,HoweIMetYourMother,Lester,1580
0,18,James vs Anthony,9867,Bad Boys For Fyfe,James,1570,189,The Full Bonty,Anthony,1536
0,18,Jordan vs Mark,33301,"What's Up, Doch?",Jordan,1532,186,GetRichOrDieTryin,Mark,1758


In [14]:
df_fixture_results.to_csv('outputs/fixture_results.csv'.format(round_number))

### Rearrange fixture data to display by team + opposition scores
- Generate team rank for that week + opponent rank for that week

In [15]:
df_home_fixture_results = df_fixture_results
df_away_fixture_results = df_fixture_results

home_renamed_cols = {
     'home_team_id': 'team_id',
     'home_team_name': 'team_name',
     'home_team_coach': 'team_coach',
     'home_team_points': 'team_points',
     'away_team_id': 'opposition_team_id',
     'away_team_name': 'opposition_team_name',
     'away_team_coach': 'opposition_team_coach',
     'away_team_points': 'opposition_team_points'
}

away_renamed_cols = {
     'home_team_id': 'opposition_team_id',
     'home_team_name': 'opposition_team_name',
     'home_team_coach': 'opposition_team_coach',
     'home_team_points': 'opposition_team_points',
     'away_team_id': 'team_id',
     'away_team_name': 'team_name',
     'away_team_coach': 'team_coach',
     'away_team_points': 'team_points'
}

df_home_fixture_results = df_home_fixture_results.rename(columns=home_renamed_cols)
df_away_fixture_results = df_away_fixture_results.rename(columns=away_renamed_cols)
df_fixture_by_each_team = pd.concat([df_home_fixture_results, df_away_fixture_results])
df_fixture_by_each_team['Rank'] = df_fixture_by_each_team.groupby('round_number')['team_points'].rank(ascending=False)
df_fixture_by_each_team['Opponent Rank'] = df_fixture_by_each_team.groupby('round_number')['opposition_team_points'].rank(ascending=False)
df_fixture_by_each_team = df_fixture_by_each_team.sort_values(['round_number', 'Rank'])
df_fixture_by_each_team.to_csv('outputs/fixture_results_by_team.csv')
df_fixture_by_each_team

,round_number,match_name,team_id,team_name,team_coach,team_points,opposition_team_id,opposition_team_name,opposition_team_coach,opposition_team_points,Rank,Opponent Rank
0,1,Lester vs Simon,9806,HoweIMetYourMother,Lester,1827,9856,It's always Sonny,Simon,1476,1.0,8.0
0,1,James vs Jordan,33301,"What's Up, Doch?",Jordan,1721,9867,Bad Boys For Fyfe,James,1623,2.0,7.0
0,1,Anthony vs Luke,5524,The Dragon's Laird,Luke,1720,189,The Full Bonty,Anthony,1662,3.0,5.0
0,1,Paul vs Mark,182,Coronaviney,Paul,1686,186,GetRichOrDieTryin,Mark,1657,4.0,6.0
0,1,Anthony vs Luke,189,The Full Bonty,Anthony,1662,5524,The Dragon's Laird,Luke,1720,5.0,3.0
...,...,...,...,...,...,...,...,...,...,...,...,...
0,18,Simon vs Luke,5524,The Dragon's Laird,Luke,1692,9856,It's always Sonny,Simon,1700,4.0,3.0
0,18,Paul vs Lester,9806,HoweIMetYourMother,Lester,1580,182,Coronaviney,Paul,1843,5.0,1.0
0,18,James vs Anthony,9867,Bad Boys For Fyfe,James,1570,189,The Full Bonty,Anthony,1536,6.0,7.0
0,18,James vs Anthony,189,The Full Bonty,Anthony,1536,9867,Bad Boys For Fyfe,James,1570,7.0,6.0


## CURRENTLY LISTED TEAMS

In [16]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/playersStatus", headers=headers)
players_json = json.loads(s.text)
df_players = pd.DataFrame.from_dict(players_json, orient='index')
df_players = df_players.merge(df_player_list, on='player_id', how='left')
df_currently_listed_players = df_players.loc[df_players['trade_status'] == "owner"]
df_currently_listed_players = df_currently_listed_players.merge(df_coaches, left_on = 'user_team_id', right_on='coach_team_id', how='left')
df_currently_listed_players.to_csv('outputs/current_teams.csv')
df_currently_listed_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position,coach_id,coach_team_id,coach_first_name,coach_team_name
0,151,owner,186.0,NaN,293957,Brodie,Grundy,COL,RUC,21562,186,Mark,GetRichOrDieTryin
1,469,owner,33301.0,NaN,290528,Max,Gawn,MEL,RUC,180650,33301,Jordan,"What's Up, Doch?"
2,755,owner,33301.0,NaN,295467,Jack,Macrae,WBD,MID,180650,33301,Jordan,"What's Up, Doch?"
3,76,owner,5524.0,NaN,293535,Lachie,Neale,BRL,MID,45168,5524,Luke,The Dragon's Laird
4,244,owner,9867.0,NaN,291570,Nat,Fyfe,FRE,MID,71052,9867,James,Bad Boys For Fyfe
...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,124,owner,9867.0,NaN,298290,Marc,Pittonet,CAR,RUC,71052,9867,James,Bad Boys For Fyfe
172,645,owner,182.0,NaN,992374,Dan,Butler,STK,FWD,300791,182,Paul,Coronaviney
173,263,owner,5524.0,NaN,1009420,Caleb,Serong,FRE,MID,45168,5524,Luke,The Dragon's Laird
174,144,owner,186.0,NaN,990291,Darcy,Cameron,COL,RUC FWD,21562,186,Mark,GetRichOrDieTryin


## CURRENT FREE AGENTS + WAIVER

In [17]:
df_available_players = df_players.loc[df_players['trade_status'] != "owner"]
df_available_players

,player_id,trade_status,user_team_id,waiver_until,feed_id,first_name,last_name,team,position
32,513,waiver,0.0,2020-09-23 04:00:00,280921,Ben,Cunnington,NTH,MID
35,628,waiver,0.0,2020-09-23 04:00:00,290627,Dion,Prestia,RIC,MID
37,809,waiver,0.0,2020-09-23 04:00:00,280078,Luke,Shuey,WCE,MID
38,270,waiver,0.0,2020-09-23 04:00:00,281007,Michael,Walters,FRE,FWD MID
47,108,waiver,0.0,2020-09-23 04:00:00,261299,Matthew,Kreuzer,CAR,RUC
...,...,...,...,...,...,...,...,...,...
821,821,waiver,0.0,2020-09-23 04:00:00,1016097,Henry,Crauford,ESS,RUC
822,822,waiver,0.0,2020-09-23 04:00:00,1020224,Tom,Hird,ESS,MID
823,825,waiver,0.0,2020-09-23 04:00:00,297668,Nicholas,Reid,WCE,FWD
824,208,waiver,0.0,2020-09-23 04:00:00,997846,Conor,*** McKenna,ESS,DEF


In [18]:
df_available_players.to_csv('outputs/current_free_agents_waiver.csv')

# Transactions

## Waiver Picks

In [19]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/processedWaivers", headers=headers)
waivers_json = json.loads(s.text)
df_waiver_transactions = pd.json_normalize(waivers_json)

wanted_columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
]

df_waiver_transactions = df_waiver_transactions.loc[:, wanted_columns]
df_waiver_transactions['source'] = "Waiver"
df_waiver_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
0,2,182,2020-03-24 04:00:02,227,296294,Blake,Acres,FRE,486,291533,Tom,McDonald,MEL,Waiver
1,2,9856,2020-03-24 04:00:02,734,293651,Hayden,Crozier,WBD,577,293987,Jarrod,Lienert,PTA,Waiver
2,2,182,2020-03-24 04:00:02,593,294318,Ollie,Wines,PTA,802,281373,Jack,Redden,WCE,Waiver
3,2,9856,2020-03-24 04:00:02,797,271045,Nic,Naitanui,WCE,356,298419,Brandan,Parfitt,GEE,Waiver
4,2,182,2020-03-24 04:00:02,159,992010,Brayden,Maynard,COL,278,998129,Will,Brodie,GCS,Waiver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
155,17,9806,2020-09-12 04:00:02,740,1004592,Tim,English,WBD,197,290629,Dyson,Heppell,ESS,Waiver
156,18,9856,2020-09-16 04:00:02,439,261323,Ben,McEvoy,HAW,452,998114,Jack,Scrimshaw,HAW,Waiver
157,18,186,2020-09-16 04:00:02,690,992242,Jordan,Dawson,SYD,416,280744,Luke,Breust,HAW,Waiver
158,18,9856,2020-09-16 04:00:02,325,220001,Gary,Ablett,GEE,586,993979,Sam,Powell-Pepper,PTA,Waiver


In [20]:
df_waiver_transactions.to_csv('outputs/waiver_transactions.csv')

## Free Agent trades

In [21]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/trades", headers=headers)
free_agent_trades_json = json.loads(s.text)
df_fa_transactions = pd.json_normalize(free_agent_trades_json)

wanted_columns = [
    'round',
    'user_id',
    'action_time',
    'buy_player.id',
    'buy_player.feed_id',
    'buy_player.first_name',
    'buy_player.last_name',
    'buy_player.team.abbrev',
    'sell_player.id',
    'sell_player.feed_id',
    'sell_player.first_name',
    'sell_player.last_name',
    'sell_player.team.abbrev',
]

df_fa_transactions = df_fa_transactions.loc[:, wanted_columns]
df_fa_transactions['source'] = "Free Agent"
df_fa_transactions

,round,user_id,action_time,buy_player.id,buy_player.feed_id,buy_player.first_name,buy_player.last_name,buy_player.team.abbrev,sell_player.id,sell_player.feed_id,sell_player.first_name,sell_player.last_name,sell_player.team.abbrev,source
0,2,182,2020-06-11 18:47:54,606,270896,Trent,Cotchin,RIC,675.0,280933,Dylan,Roberton,STK,Free Agent
1,3,182,2020-06-21 19:41:30,564,1006096,Xavier,Duursma,PTA,579.0,1004998,Todd,Marshall,PTA,Free Agent
2,3,182,2020-06-21 19:39:37,550,297354,Karl,Amon,PTA,214.0,993817,Darcy,Parish,ESS,Free Agent
3,3,182,2020-06-20 23:20:43,579,1004998,Todd,Marshall,PTA,227.0,296294,Blake,Acres,FRE,Free Agent
4,3,182,2020-06-20 15:04:59,228,295067,James,Aish,FRE,199.0,270588,Cale,Hooker,ESS,Free Agent
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
252,13,33301,2020-08-19 06:03:06,798,298298,Jackson,Nelson,WCE,311.0,1001195,Izak,Rankine,GCS,Free Agent
253,16,33301,2020-09-05 04:26:32,274,998130,Ben,Ainsworth,GCS,5.0,294307,Brad,Crouch,ADE,Free Agent
254,16,33301,2020-09-05 19:29:38,767,291545,Mitch,Wallis,WBD,429.0,294077,Michael,Hartley,HAW,Free Agent
255,16,33301,2020-09-05 19:33:29,281,295446,Sam,Collins,GCS,767.0,291545,Mitch,Wallis,WBD,Free Agent


In [22]:
df_fa_transactions.to_csv('outputs/free_agent_transactions.csv')

## Team Trades

In [23]:
s = session.get("https://supercoach.heraldsun.com.au/2020/api/afl/draft/v1/leagues/96/teamtrades", headers=headers)
team_trades_json = json.loads(s.text)

In [24]:
df_trades_player_list = pd.DataFrame()

for trade in team_trades_json:
    df_lines = pd.json_normalize(trade['team_trade_players'])
    df_lines['round'] = trade['round']
    df_lines['user_team_id'] = trade['user_team_id']
    df_lines['processed'] = trade['action_time']
    df_lines['status'] = trade['status_id']
    df_lines['source'] = 'Trade'
    df_trades_player_list = pd.concat([df_trades_player_list,df_lines])
df_trades_player_list

,id,trade_id,player_id,trade_player_id,round,user_team_id,processed,status,source
0,26827,16727,214,322,2,182,2020-06-12 10:01:48,2,Trade
0,30776,19019,522,574,3,186,2020-06-15 20:17:51,7,Trade
0,40668,24785,395,752,4,182,2020-07-07 00:31:22,3,Trade
0,49794,30082,90,755,5,33301,2020-06-30 21:43:08,3,Trade
1,49795,30082,570,455,5,33301,2020-06-30 21:43:08,3,Trade
...,...,...,...,...,...,...,...,...,...
1,103218,61739,356,0,15,186,2020-09-02 19:09:30,7,Trade
0,103293,61793,72,593,15,9867,2020-09-04 08:52:23,1,Trade
0,103299,61799,136,729,15,9856,2020-09-04 19:44:53,7,Trade
0,103517,61956,633,460,16,189,2020-09-05 10:37:30,7,Trade


In [25]:
df_team_one_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'player_id', 'source']]
df_team_two_trades_player_list = df_trades_player_list.loc[df_trades_player_list['status'] == 7, ['round', 'user_team_id', 'processed', 'trade_player_id', 'source']]
df_team_two_trades_player_list = df_team_two_trades_player_list.rename(columns={'trade_player_id': 'player_id'})
df_trades_by_player = pd.concat([df_team_one_trades_player_list, df_team_two_trades_player_list]).sort_values(by=['processed'])
df_trades_by_player = df_trades_by_player.merge(df_player_list[['player_id', 'feed_id', 'first_name', 'last_name', 'team']], on='player_id', how='left')
df_trades_by_player = df_trades_by_player.rename(columns={'feed_id': 'player.feed_id', 'first_name': 'player.first_name', 'last_name': 'player.last_name', 'team': 'player.team.abbrev'})
df_trades_by_player

,round,user_team_id,processed,player_id,source,player.feed_id,player.first_name,player.last_name,player.team.abbrev
0,3,186,2020-06-15 20:17:51,522,Trade,260227,Shaun,Higgins,NTH
1,3,186,2020-06-15 20:17:51,574,Trade,994295,Dan,Houston,PTA
2,5,9856,2020-07-01 21:22:52,328,Trade,296733,Mark,Blicavs,GEE
3,5,9856,2020-07-01 21:22:52,637,Trade,294674,Nick,Vlastuin,RIC
4,5,9856,2020-07-01 21:22:52,389,Trade,295265,Nick,Haynes,GWS
5,5,9856,2020-07-01 21:22:52,90,Trade,261224,Dayne,Zorko,BRL
6,5,9856,2020-07-01 21:22:52,491,Trade,298210,Christian,Petracca,MEL
7,5,9856,2020-07-01 21:22:52,335,Trade,270917,Patrick,Dangerfield,GEE
8,5,9856,2020-07-01 21:22:52,755,Trade,295467,Jack,Macrae,WBD
9,5,9856,2020-07-01 21:22:52,715,Trade,290307,Dane,Rampe,SYD


## Combined Free Agents, Waiver, and Trades transactions list

In [26]:

columns = [
    'round',
    'user_team_id',
    'processed',
    'player_id',
    'player.feed_id',
    'player.first_name',
    'player.last_name',
    'player.team.abbrev',
    'dropped_player_id',
    'dropped_player.feed_id',
    'dropped_player.first_name',
    'dropped_player.last_name',
    'dropped_player.team.abbrev',
    'source',
]
df_waiver_transactions.columns = columns
df_fa_transactions.columns = columns

df_transactions = pd.concat([df_waiver_transactions, df_fa_transactions, df_trades_by_player])
df_transactions = df_transactions.sort_values(by=['processed'])
df_transactions

,round,user_team_id,processed,player_id,player.feed_id,player.first_name,player.last_name,player.team.abbrev,dropped_player_id,dropped_player.feed_id,dropped_player.first_name,dropped_player.last_name,dropped_player.team.abbrev,source
0,2,182,2020-03-24 04:00:02,227,296294,Blake,Acres,FRE,486.0,291533,Tom,McDonald,MEL,Waiver
1,2,9856,2020-03-24 04:00:02,734,293651,Hayden,Crozier,WBD,577.0,293987,Jarrod,Lienert,PTA,Waiver
2,2,182,2020-03-24 04:00:02,593,294318,Ollie,Wines,PTA,802.0,281373,Jack,Redden,WCE,Waiver
3,2,9856,2020-03-24 04:00:02,797,271045,Nic,Naitanui,WCE,356.0,298419,Brandan,Parfitt,GEE,Waiver
4,2,182,2020-03-24 04:00:02,159,992010,Brayden,Maynard,COL,278.0,998129,Will,Brodie,GCS,Waiver
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
204,18,9856,2020-09-20 12:09:08,586,993979,Sam,Powell-Pepper,PTA,277.0,998260,Jack,Bowes,GCS,Free Agent
205,18,9856,2020-09-20 15:19:10,739,290085,Taylor,Duryea,WBD,365.0,270912,Jack,Steven,GEE,Free Agent
82,18,186,2020-09-20 15:19:24,144,990291,Darcy,Cameron,COL,690.0,992242,Jordan,Dawson,SYD,Free Agent
206,18,9856,2020-09-20 18:08:54,559,991930,Darcy,Byrne-Jones,PTA,739.0,290085,Taylor,Duryea,WBD,Free Agent


In [27]:
df_transactions.to_csv('outputs/transactions.csv')

## Add source to player match results

In [28]:
def get_transaction_status(row):
    post_transaction_list = pd.DataFrame()
    round_number = row['round']
    feed_id = row['player.feed_id']
    source = row['source']
    player_rows = df_player_match_results.loc[(df_player_match_results['round'] >= round_number) & (df_player_match_results['feed_id'] == feed_id), :]
    player_rows['source'] = source
    if len(player_rows) > 0:
        df_player_match_results.update(player_rows)

df_player_match_results['source'] = "Draft"
df_transactions.apply(lambda row: get_transaction_status(row), axis=1)
df_player_match_results

C:\Users\richardm\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,round,coach_id,coach_team_id,coach_first_name,coach_team_name,player_id,played_position,points,on_field,feed_id,first_name,last_name,team,position,round_number_alt,source
0,1.0,86078.0,9806.0,Lester,HoweIMetYourMother,153.0,DEF,156.0,True,291313,Jeremy,Howe,COL,DEF,R1,Draft
1,2.0,86078.0,9806.0,Lester,HoweIMetYourMother,153.0,DEF,126.0,True,291313,Jeremy,Howe,COL,DEF,R2,Draft
2,3.0,86078.0,9806.0,Lester,HoweIMetYourMother,153.0,DEF,88.0,True,291313,Jeremy,Howe,COL,DEF,R3,Draft
3,4.0,86078.0,9806.0,Lester,HoweIMetYourMother,153.0,DEF,112.0,True,291313,Jeremy,Howe,COL,DEF,R4,Draft
4,1.0,86078.0,9806.0,Lester,HoweIMetYourMother,789.0,DEF,152.0,True,240283,Shannon,Hurn,WCE,DEF,R1,Draft
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3163,18.0,71052.0,9867.0,James,Bad Boys For Fyfe,65.0,DEF,57.0,False,291548,Ryan,Lester,BRL,DEF,R18,Free Agent
3164,16.0,300791.0,182.0,Paul,Coronaviney,804.0,DEF,50.0,True,997823,Josh,Rotham,WCE,DEF,R16,Free Agent
3165,17.0,300791.0,182.0,Paul,Coronaviney,804.0,DEF,31.0,True,997823,Josh,Rotham,WCE,DEF,R17,Free Agent
3166,18.0,300791.0,182.0,Paul,Coronaviney,804.0,DEF,45.0,False,997823,Josh,Rotham,WCE,DEF,R18,Free Agent


In [29]:
df_player_match_results.to_csv('outputs/player_match_results.csv')